Let us go through a full worked example.  Here we have a Tipsy SPH dataset.  By general
inspection, we see that there are stars present in the dataset, since
there are fields with field type: `Stars` in the `ds.field_list`. Let's look 
at the `derived_field_list` for all of the `Stars` fields. 

In [ ]:
import yt
import numpy as np

ds = yt.load("TipsyGalaxy/galaxy.00300")
for field in ds.derived_field_list:
    if field[0] == 'Stars':
        print (field)

We will filter these into young stars and old stars by masking on the ('Stars', 'creation_time') field. 

In order to do this, we first make a function which applies our desired cut.  This function must accept two arguments: `pfilter` and `data`.  The first argument is a `ParticleFilter` object that contains metadata about the filter its self.  The second argument is a yt data container.

Let's call "young" stars only those stars with ages less 5 million years.  Since Tipsy assigns a very large `creation_time` for stars in the initial conditions, we need to also exclude stars with negative ages. 

Conversely, let's define "old" stars as those stars formed dynamically in the simulation with ages greater than 5 Myr.  We also include stars with negative ages, since these stars were included in the simulation initial conditions.

We make use of `pfilter.filtered_type` so that the filter definition will use the same particle type as the one specified in the call to `add_particle_filter` below.  This makes the filter definition usable for arbitrary particle types.  Since we're only filtering the `"Stars"` particle type in this example, we could have also replaced `pfilter.filtered_type` with `"Stars"` and gotten the same result.

In [ ]:
def young_stars(pfilter, data):
    age = data.ds.current_time - data[pfilter.filtered_type, "creation_time"]
    filter = np.logical_and(age.in_units('Myr') <= 5, age >= 0)
    return filter

def old_stars(pfilter, data):
    age = data.ds.current_time - data[pfilter.filtered_type, "creation_time"]
    filter = np.logical_or(age.in_units('Myr') >= 5, age < 0)
    return filter

Now we define these as particle filters within the yt universe with the
`add_particle_filter()` function.

In [ ]:
yt.add_particle_filter("young_stars", function=young_stars, filtered_type='Stars', requires=["creation_time"])

yt.add_particle_filter("old_stars", function=old_stars, filtered_type='Stars', requires=["creation_time"])

Let us now apply these filters specifically to our dataset.

Let's double check that it worked by looking at the derived_field_list for any new fields created by our filter.

In [ ]:
ds.add_particle_filter('young_stars')
ds.add_particle_filter('old_stars')

for field in ds.derived_field_list:
    if "young_stars" in field or "young_stars" in field[1]:
        print (field)

We see all of the new `young_stars` fields as well as the 4 deposit fields.  These deposit fields are `mesh` fields generated by depositing particle fields on the grid.  Let's generate a couple of projections of where the young and old stars reside in this simulation by accessing some of these new fields.

In [ ]:
p = yt.ProjectionPlot(ds, 'z', [('deposit', 'young_stars_cic'), ('deposit', 'old_stars_cic')], width=(40, 'kpc'), center='m')
p.set_figure_size(5)
p.show()

We see that young stars are concentrated in regions of active star formation, while old stars are more spatially extended.